In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkFiles
import json
import requests
import sys

In [ ]:
spark = SparkSession.builder \
.master("local") \
.appName("Exercise2") \
.getOrCreate()

In [ ]:
df = spark.read.csv("exampleData.csv", header=True, inferSchema=True)

In [ ]:
staticSchema = df.schema

In [ ]:
streamingDF = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("/home/jovyan/*.csv")

In [ ]:
streamingDF.isStreaming

In [ ]:
filteredDF = streamingDF.filter((F.col('air_temperature')<5))

In [ ]:
df = filteredDF.writeStream.format("memory") \
.queryName("weatherQuery") \
.outputMode("append") \
.start()

In [ ]:
df.awaitTermination() # run and terminate by Keyboard Interruption (interrupt the kernel - square button)

In [ ]:
spark.sql("""
SELECT *
FROM weatherQuery
""")\
.show(5)

## Read from URL 

In [ ]:
url = "https://people.sc.fsu.edu/~jburkardt/data/csv/homes.csv"
# try your url here

In [ ]:
spark.sparkContext.addFile(url)

In [ ]:
df = spark.read.csv("file://"+SparkFiles.get('homes.csv'), header=True, inferSchema=True) # change home.csv to the filename according to your data

In [ ]:
df.show(3)

## Read from API (open/public)

In [ ]:
API_URL = "https://jsonplaceholder.typicode.com/posts" # try your url here

In [ ]:
response = requests.get(API_URL)

In [ ]:
if response.status_code == 200:
    # Convert the JSON response to dictionary
    data = json.loads(response.text)
    
    # Create df from data
    df = spark.createDataFrame(data)
    
    # Show df
    df.show(3)
else:
    print("Failed to retrieve data from API")
